<a href="https://colab.research.google.com/github/tsudiacr/PruebaDiana/blob/master/copy_of_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# prototype 1 // cpu


MODELADO


In [ ]:
#Importar bibliotecas necesarias
!pip install transformers
!pip install torch
!pip install tensorflow

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

import torch
from torch.utils.data import DataLoader, TensorDataset, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup

import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
import pandas as pd

# Montar Google Drive
drive.mount('/content/drive', force_remount=True)

# Cargar el dataset preprocesado desde Google Drive
file_path = '/content/drive/My Drive/usa_2020_normalizado.csv'
usa_2020 = pd.read_csv(file_path)

# Reducción de dataset para entrenamiento
usa_2020 = usa_2020.sample(frac=0.01, random_state=42)

# Guardar el dataset reducido en un archivo CSV en Colab
usa_2020.to_csv("usa_2020_reducido.csv", index=False)
print("Dataset reducido guardado en Colab.")

# Descargar el archivo CSV reducido desde Colab
from google.colab import files
files.download("usa_2020_reducido.csv")

# Mostrar información del dataset reducido
print(f"Tamaño del dataset reducido: {usa_2020.shape}")


Mounted at /content/drive
Dataset reducido guardado en Colab.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Tamaño del dataset reducido: (164207, 11)


In [ ]:
import re

def clean_scoring_string(text):
    # Eliminar números y otros caracteres no alfabéticos
    cleaned_text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Eliminar espacios en blanco adicionales
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    return cleaned_text.strip()

# Aplicar la función de limpieza a la columna "Scoring String"
usa_2020['Cleaned Scoring String'] = usa_2020['Scoring String'].apply(clean_scoring_string)

#Actualizamos Score para normalizar a valores 0 (Score < 0) y 1 (Score > 0)
usa_2020['Score'] = usa_2020['Score'].apply(lambda x: 0 if x < 0 else 1)

# Verificar los cambios
print(usa_2020[['Scoring String', 'Cleaned Scoring String']].head())


                                       Scoring String Cleaned Scoring String
2119160                             postponed (-0.21)              postponed
4603439                                 wrong (-0.54)                  wrong
2454579   grants (0.23)  support (0.44)  legal (0.13)   grants support legal
7421914                          embarrassing (-0.41)           embarrassing
6166657  hero (0.67)  legal (0.13)  relentless (0.05)  hero legal relentless


In [ ]:
from operator import mod
#Preparar datos para BERT // tokenización y creación de dataset personalizado:
# Inicializar el tokenizer de BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# Función para tokenizar el texto
def tokenize_sentences(sentences, max_length):
    input_ids = []
    attention_masks = []
    i=0
    for sentence in sentences:
        encoded_dict = tokenizer.encode_plus(
            sentence,                      # Oración a codificar
            add_special_tokens = True,     # Agregar '[CLS]' y '[SEP]'
            max_length = max_length,       # Longitud máxima
            #pad_to_max_length = True,      # Padding
            return_attention_mask = True,  # Retornar máscara de atención
            return_tensors = 'pt',         # Retornar tensores de PyTorch
            truncation=True,
            padding='max_length'
        )

        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])

    return torch.cat(input_ids, dim=0), torch.cat(attention_masks, dim=0)

# Tokenización de los textos
input_ids, attention_masks = tokenize_sentences(usa_2020['Cleaned Scoring String'].values, max_length=64)

# Convertir las etiquetas de sentimiento a tensores
labels = torch.tensor(usa_2020['Score'].values)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
# Dividir en conjuntos de entrenamiento y prueba
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, test_size=0.2, random_state=42)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids, test_size=0.2, random_state=42)

# Crear tensores de datos
train_data = TensorDataset(train_inputs, train_masks, train_labels)
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)

# Crear DataLoader
batch_size = 32

train_dataloader = DataLoader(
    train_data,
    sampler = RandomSampler(train_data),
    batch_size = batch_size
)

validation_dataloader = DataLoader(
    validation_data,
    sampler = SequentialSampler(validation_data),
    batch_size = batch_size
)

In [ ]:
# Cargar el modelo BERT preentrenado
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = 2,  # Número de etiquetas de clasificación
    output_attentions = False,
    output_hidden_states = False,
)

# Configurar el optimizador
optimizer = AdamW(model.parameters(), lr = 2e-5, eps = 1e-8)

# Número de épocas
epochs = 2

# Total de pasos de entrenamiento
total_steps = len(train_dataloader) * epochs

# Crear el programador de tasa de aprendizaje
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = total_steps)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Función de evaluación
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


# Función de entrenamiento
def train_model():
    for epoch in range(epochs):
        print(f'Epoch {epoch + 1}/{epochs}')

        # Entrenamiento
        model.train()
        total_loss = 0

        for step, batch in enumerate(train_dataloader):
            b_input_ids, b_input_mask, b_labels = batch
            b_input_ids = b_input_ids.to(device)
            b_input_mask = b_input_mask.to(device)
            b_labels = b_labels.to(device).type(torch.long)

            model.zero_grad()

            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
            loss = outputs.loss
            total_loss += loss.item()

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()

        avg_train_loss = total_loss / len(train_dataloader)
        print(f"Average training loss: {avg_train_loss}")

        # Evaluación
        model.eval()
        eval_accuracy = 0
        nb_eval_steps = 0

        for batch in validation_dataloader:
            b_input_ids, b_input_mask, b_labels = batch
            b_input_ids = b_input_ids.to(device)
            b_input_mask = b_input_mask.to(device)
            b_labels = b_labels.to(device)

            with torch.no_grad():
                outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

            logits = outputs.logits
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()

            eval_accuracy += flat_accuracy(logits, label_ids)
            nb_eval_steps += 1

        print(f"Validation accuracy: {eval_accuracy / nb_eval_steps}")


train_model()


Epoch 1/2
Average training loss: 0.09887596869253854
Validation accuracy: 0.9821324245374878
Epoch 2/2
Average training loss: 0.04029737193445117
Validation accuracy: 0.9889849074975657


In [ ]:
# Realizar predicciones en el conjunto de validación
model.eval()
predictions, true_labels = [], []

for batch in validation_dataloader:
    b_input_ids, b_input_mask, b_labels = batch
    with torch.no_grad():
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

    logits = outputs.logits
    predictions.append(logits.detach().cpu().numpy())
    true_labels.append(b_labels.to('cpu').numpy())

# Aplanar predicciones y etiquetas reales
flat_predictions = [item for sublist in predictions for item in sublist]
flat_true_labels = [item for sublist in true_labels for item in sublist]

# Calcular y mostrar el reporte de clasificación
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
print(classification_report(flat_true_labels, flat_predictions))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99     15665
           1       0.99      0.99      0.99     17177

    accuracy                           0.99     32842
   macro avg       0.99      0.99      0.99     32842
weighted avg       0.99      0.99      0.99     32842



In [ ]:
# Guardar el modelo en Google Drive
model_save_path = '/content/drive/My Drive/bert_model_cpu.bin'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

Model saved to /content/drive/My Drive/bert_model_cpu.bin


In [ ]:
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Cargar el modelo BERT preentrenado
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = 2,  # Número de etiquetas de clasificación
    output_attentions = False,
    output_hidden_states = False,
)

# Mover el modelo a la CPU
device = torch.device("cpu")
model.to(device)

# Cargar el estado del modelo guardado
model.load_state_dict(torch.load(model_save_path, map_location=device))
print("Model loaded from drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded from drive


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Cargar el modelo y el tokenizer
model_path = '/content/drive/My Drive/bert_model_cpu.bin'
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Cargar el estado del modelo guardado
device = torch.device("cpu")
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()

def preprocess_tweet(tweet, tokenizer, max_length=64):
    """Tokenize and preprocess the tweet."""
    encoded_dict = tokenizer.encode_plus(
        tweet,
        add_special_tokens=True,  # Agregar '[CLS]' y '[SEP]'
        max_length=max_length,    # Longitud máxima
        pad_to_max_length=True,   # Padding
        truncation=True,          # Truncar ejemplos a longitud máxima
        return_attention_mask=True,  # Retornar máscara de atención
        return_tensors='pt',      # Retornar tensores de PyTorch
    )
    return encoded_dict['input_ids'], encoded_dict['attention_mask']

def predict_tweet_sentiment(tweet, model, tokenizer):
    """Predict the sentiment of a tweet."""
    input_ids, attention_mask = preprocess_tweet(tweet, tokenizer)

    # Mover tensores a la CPU
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)

    # Realizar la predicción
    with torch.no_grad():
        outputs = model(input_ids, token_type_ids=None, attention_mask=attention_mask)

    logits = outputs.logits
    prediction = torch.argmax(logits, dim=1).item()
    return "Positivo" if prediction == 1 else "Negativo"

# Probar el modelo con un tweet específico
tweet = "The elections sucked"
print(f"Tweet: {tweet}")
print(f"Sentimiento predicho: {predict_tweet_sentiment(tweet, model, tokenizer)}")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Tweet: The elections sucked
Sentimiento predicho: Negativo
